# 📘 Model Predictive Control (MPC) for Biogas Optimization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/benmola/OpenAD-lib/blob/main/notebooks/05_MPC_Control.ipynb)

This notebook demonstrates **Model Predictive Control (MPC)** for optimizing biogas production in anaerobic digesters using the AM2 model.

---

## 📚 References

- **Data-Driven Modeling & Control**: [Dekhici et al. (2024) - ResearchGate](https://www.researchgate.net/publication/378298857_Data-Driven_Modeling_Order_Reduction_and_Control_of_Anaerobic_Digestion_Processes)
- **do-mpc Framework**: [do-mpc Documentation](https://www.do-mpc.com)

## 🔬 MPC Background

### Model Predictive Control

MPC solves an optimization problem at each time step:

$$\min_{u_0, ..., u_{N-1}} \sum_{k=0}^{N-1} l(x_k, u_k) + V_f(x_N)$$

Subject to:
- $x_{k+1} = f(x_k, u_k)$ (system dynamics)
- $u_{min} \leq u_k \leq u_{max}$ (input constraints)
- $x_{min} \leq x_k \leq x_{max}$ (state constraints)

### Application to AD Control

For biogas optimization:

**Control Inputs:**
- $D$ = Dilution rate (feeding rate / volume)
- $S_{1,in}$ = Influent substrate concentration

**Objectives:**
1. **Maximize biogas**: $\max Q = k_6 \mu_2(S_2) X_2$
2. **VFA tracking**: $\min |S_2 - S_{2,ref}|$ (process stability)

### MPC Formulation for Biogas Maximization

$$\max_{D, S_{1,in}} \sum_{k=0}^{N-1} Q_k$$

Subject to:
$$0.1 \leq D \leq 0.5 \quad \text{(d}^{-1}\text{)}$$
$$5 \leq S_{1,in} \leq 30 \quad \text{(g COD/L)}$$
$$S_2 \leq S_{2,max} \quad \text{(VFA constraint for stability)}$$

## 1️⃣ Setup (Google Colab)

In [ ]:
# Install OpenAD-lib with control support (uncomment for Colab)
# !pip install git+https://github.com/benmola/OpenAD-lib.git

import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if not IN_COLAB:
    sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

print(f"Running in Colab: {IN_COLAB}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from openad_lib.models.mechanistic import AM2Model, AM2Parameters
from openad_lib.control import AM2MPCController

print("✅ All imports successful!")

## 2️⃣ Initialize AM2 Model and MPC Controller

In [ ]:
# Initialize AM2 model with calibrated parameters
params = AM2Parameters(
    m1=0.18,    # Acidogenic growth rate
    K1=20.0,    # Half-saturation for S1
    m2=0.15,    # Methanogenic growth rate
    Ki=30.0,    # Inhibition constant
    K2=45.0     # Half-saturation for S2
)

model = AM2Model(params=params)

print("📊 AM2 Model initialized")
print(f"   States: S1, S2, X1, X2")
print(f"   Outputs: Q (biogas rate)")

In [ ]:
# Initialize MPC controller
controller = AM2MPCController(model)

# Configure MPC for biogas maximization
controller.setup_controller(
    objective='maximize_biogas',
    prediction_horizon=10,
    control_horizon=5,
    dt=1.0  # 1 day time step
)

print("🎮 MPC Controller configured")
print("   Objective: Maximize biogas production")
print("   Prediction horizon: 10 days")
print("   Control horizon: 5 days")

## 3️⃣ Run MPC Simulation

We'll run a 50-day simulation where MPC optimizes the control inputs to maximize biogas production while maintaining process stability.

In [ ]:
# Initial state [S1, S2, X1, X2]
x0 = np.array([5.0, 2.0, 0.5, 0.3])

# Run closed-loop simulation
print("🚀 Running MPC closed-loop simulation (50 days)...")
history = controller.run_closed_loop(
    x0=x0,
    n_steps=50,
    verbose=True
)

print(f"\n✅ Simulation complete!")
print(f"   Total biogas produced: {sum(history['Q']):.2f} L")
print(f"   Average production rate: {np.mean(history['Q']):.3f} L/d")

## 4️⃣ Visualize Results

In [ ]:
plt.style.use('bmh')
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

# Plot 1: VFA Concentration (S2)
ax = axes[0]
ax.plot(history['time'], history['S2'], 'b-', linewidth=2, label='VFA (S2)')
ax.set_ylabel('VFA [g COD/L]', fontsize=14, fontweight='bold')
ax.set_title('VFA Concentration (Process Stability Indicator)', fontsize=16, pad=10)
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)

# Plot 2: Biogas Production
ax = axes[1]
ax.plot(history['time'], history['Q'], 'g-', linewidth=2, label='Biogas Rate')
ax.set_ylabel('Q [L/d]', fontsize=14, fontweight='bold')
ax.set_title('Biogas Production Rate (MPC Objective)', fontsize=16, pad=10)
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)

# Plot 3: Control Input (D)
ax = axes[2]
ax.step(history['time'], history['D'], 'r-', where='post', linewidth=2, label='Dilution Rate (D)')
ax.set_ylabel('D [1/d]', fontsize=14, fontweight='bold')
ax.set_xlabel('Time [days]', fontsize=14, fontweight='bold')
ax.set_title('Optimal Control Input', fontsize=16, pad=10)
ax.set_ylim(0, 0.6)
ax.axhline(y=0.1, color='gray', linestyle='--', alpha=0.5, label='Lower bound')
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Upper bound')
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## 5️⃣ Alternative: VFA Setpoint Tracking

For process stability, we can also use MPC for VFA setpoint tracking:

$$\min_{D} \sum_{k=0}^{N-1} (S_{2,k} - S_{2,ref})^2$$

In [ ]:
# Reconfigure for VFA tracking
controller.setup_controller(
    objective='track_vfa',
    vfa_setpoint=2.0,  # Target VFA = 2.0 g COD/L
    prediction_horizon=10
)

# Run simulation
print("🚀 Running VFA tracking simulation...")
history_track = controller.run_closed_loop(
    x0=np.array([5.0, 1.0, 0.5, 0.3]),
    n_steps=40
)

# Plot tracking performance
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(history_track['time'], history_track['S2'], 'b-', linewidth=2, label='VFA (S2)')
ax.axhline(y=2.0, color='r', linestyle='--', linewidth=2, label='Setpoint')
ax.set_xlabel('Time [days]', fontsize=14, fontweight='bold')
ax.set_ylabel('VFA [g COD/L]', fontsize=14, fontweight='bold')
ax.set_title('MPC VFA Setpoint Tracking', fontsize=16, pad=10)
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 📝 Summary

This notebook demonstrated:

1. **MPC Formulation** for anaerobic digestion control
2. **Biogas Maximization** - optimizing feeding strategy
3. **VFA Tracking** - maintaining process stability
4. **Constraint Handling** - respecting physical limits

### Key MPC Advantages

| Feature | Benefit |
|---------|--------|
| Predictive | Anticipates future behavior |
| Optimal | Solves optimization at each step |
| Constrained | Respects physical limits |
| Multivariable | Handles multiple inputs/outputs |

### Next Steps

- Combine with [MTGP](04_MTGP_Prediction.ipynb) for uncertainty-aware control
- Compare with open-loop control
- Try different objective weights